# Homework 4 - Part 1
## Deep Learning Technologies
### Shruti Badrinarayanan - 016768141

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torchvision.models.detection import maskrcnn_resnet50_fpn
import torchvision.transforms as T
from PIL import Image, ImageDraw
import cv2

# Load pre-trained Mask R-CNN model with COCO weights
model = maskrcnn_resnet50_fpn(pretrained=True)

# Set model to evaluation mode
model.eval()

# Define the transform to apply to the input images
transform = T.Compose([T.ToTensor()])

# Define function to perform inference on each image
def inference(image_paths, output_video_path):
    for i, image_path in enumerate(image_paths):
        # Load image
        image = Image.open(image_path).convert("RGB")
        # Apply transform
        image_tensor = transform(image)
        # Add batch dimension
        image_tensor = image_tensor.unsqueeze(0)
        # Perform inference
        with torch.no_grad():
            prediction = model(image_tensor)[0]

        # Draw bounding boxes on the image
        draw = ImageDraw.Draw(image)
        for score, label, box in zip(prediction['scores'], prediction['labels'], prediction['boxes']):
            if score > 0.5 and label == 3:  # Only consider detections with confidence above 0.5 and label corresponding to vehicles
                draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline="red")

        # Save annotated image
        annotated_image_path = f'annotated_frame_{i:04d}.jpg'
        image.save(annotated_image_path)

    # Convert annotated images to video
    images = [cv2.imread(f'annotated_frame_{i:04d}.jpg') for i in range(len(image_paths))]
    height, width, _ = images[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))
    for img in images:
        video_out.write(img)
    video_out.release()

# Example usage
num_images = 148  # Number of discrete images/frames
image_paths = ['/content/drive/MyDrive/Deep Learning/Assignments/Assignment 4/Part 1/Images/frame_{i:04d}.jpg'.format(i=i) for i in range(num_images)]
output_video_path = '/content/drive/MyDrive/Deep Learning/Assignments/Assignment 4/Part 1/inferenced_video.mp4'
inference(image_paths, output_video_path)